In [6]:
#!pip install shapely

In [5]:
#!pip install pytesseract

In [10]:
#!pip install scikit-learn

In [44]:
import os
import json
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

from shapely.geometry import Polygon
import glob
from PIL import Image
from pytesseract import pytesseract
from lxml import etree
import ast

from sklearn.model_selection import train_test_split
pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [45]:
f = open('../assets/doc-0001.json')
label_studio_data = json.load(f)

In [46]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    # print(poly_1,poly_2)
    # iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    iou = poly_1.intersection(poly_2).area
    min_area = min(poly_1.area,poly_2.area)
    return iou/min_area
    
    
def hocr_to_dataframe(fp):
    doc = etree.parse(fp)
    words = []
    wordConf = []
    coords_list = []
    for path in doc.xpath('//*'):
        if 'ocrx_word' in path.values():
            coord_text = path.values()[2].split(';')[0].split(' ')[1:] 
            word_coord = list(map(int, coord_text)) #x1, y1, x2, y2
            conf = [x for x in path.values() if 'x_wconf' in x][0]
            wordConf.append(int(conf.split('x_wconf ')[1]))
            words.append(path.text)
            coords_list.append(word_coord)

    dfReturn = pd.DataFrame({'word' : words,
                             'coords': coords_list,
                             'confidence' : wordConf})

    return(dfReturn)










In [81]:
etree.parse('./data/layoutlmv3_hocr_output/doc-0001.hocr')

In [47]:
document_data = dict()
document_data['file_name'] = []
document_data['labelled_bbox']= []

for i in range(len(label_studio_data)):
    row = label_studio_data[i]
    file_name = os.path.basename(row['data']['image'])
    label_list, labels, bboxes = [], [], []

    for label_ in row['annotations'][0]['result']:
        label_value = label_['value']
        x, y, w, h = label_value['x'], label_value['y'], label_value['width'], label_value['height']
        original_w , original_h = label_['original_width'], label_['original_height']

        x1 = int((x * original_w) / 100)
        y1 = int((y * original_h) / 100)
        x2 = x1 + int(original_w*w / 100)
        y2 = y1 + int(original_h*h / 100)
        
        label = label_value['rectanglelabels']
        label_list.append((label, (x1,y1,x2,y2), original_h, original_w))
        
    document_data['file_name'].append(file_name)    
    document_data['labelled_bbox'].append(label_list)        

custom_dataset = pd.DataFrame(document_data)





In [83]:
custom_dataset['labelled_bbox'][2]

[(['doc_number'], (1926, 298, 2165, 419), 2491, 3489),
 (['doc_date'], (1145, 489, 2310, 580), 2491, 3489),
 (['supplier_name'], (774, 702, 1361, 895), 2491, 3489),
 (['supplier_phone'], (2349, 741, 2897, 813), 2491, 3489)]

In [67]:

label2id = {"doc_number": 0, "doc_date": 1, "supplier_name": 2, "supplier_phone": 3, "receiver_tax_id": 4 }
id2label = {v:k for k, v in label2id.items()}






In [68]:

%%time

final_list = []
    
for i in tqdm(custom_dataset.iterrows(), total=custom_dataset.shape[0]):
    custom_label_text = {}
    word_list = []
    ner_tags_list  = []
    bboxes_list = []
    
    file_name = i[1]['file_name']
    for image in glob.glob('../assets/doclabel/*.jpg'): 
        frame_file_name = os.path.basename(image)
        if frame_file_name == file_name:
            custom_label_text['id'] = i[0]
            image_basename = os.path.basename(image)
            custom_label_text['file_name'] = image_basename
            annotations = []
            label_coord_list = i[1]['labelled_bbox']
            for label_coord in label_coord_list:
                (x1,y1,x2,y2) = label_coord[1]
                box1 = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]] 
                label = label_coord[0][0]
                base_name = os.path.join('./data', 'layoutlmv3_hocr_output',os.path.basename(image).split('.')[0])
                pytesseract.run_tesseract(image, base_name, extension='box', lang=None, config="hocr")
                hocr_file = os.path.join(base_name+'.hocr')
                hocr_df = hocr_to_dataframe(hocr_file)
                for word in hocr_df.iterrows():
                    coords = word[1]['coords']
                    (x1df,y1df,x2df,y2df) = coords
                    box2 = [[x1df, y1df], [x2df, y1df], [x2df, y2df], [x1df, y2df]]
                    words = word[1]['word']
                    overlap_perc = calculate_iou(box1,box2)
                    temp_dic = {}
                    if overlap_perc > 0.80:
                        if words != '-':
                            word_list.append(words)
                            bboxes_list.append(coords)
                            label_id = label2id[label]                              
                            ner_tags_list.append(label_id)
                        
                        custom_label_text['tokens'] = word_list
                        custom_label_text['bboxes'] = bboxes_list
                        custom_label_text['ner_tags'] = ner_tags_list

    final_list.append(custom_label_text)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:19<00:00,  3.29s/it]

CPU times: total: 438 ms
Wall time: 19.8 s


In [69]:
final_list

[{'id': 0,
  'file_name': 'doc-0001.jpg',
  'tokens': ['1428',
   '4',
   'aprycta',
   '2022',
   'r.',
   'Vin',
   'Opnos',
   'Anexcangp',
   'Eprenbesuy',
   '19,Ten.8-910-704-09-21',
   '711603577035'],
  'bboxes': [[1058, 270, 1146, 302],
   [1216, 271, 1238, 301],
   [1252, 245, 1403, 315],
   [1418, 270, 1507, 301],
   [1522, 277, 1547, 301],
   [775, 481, 823, 511],
   [838, 480, 960, 520],
   [972, 481, 1178, 520],
   [1193, 481, 1410, 512],
   [316, 1554, 750, 1591],
   [374, 1601, 649, 1634]],
  'ner_tags': [0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4]},
 {'id': 1,
  'file_name': 'doc-0002.jpg',
  'tokens': ['1428',
   '04.08.2022',
   'VngueugyanbHbiit',
   'npeanpvHumatenb',
   'Opnos',
   'Anexcanap',
   'Esrexbesniy',
   '711603577035'],
  'bboxes': [[1175, 962, 1282, 1000],
   [1367, 962, 1617, 1000],
   [461, 133, 790, 173],
   [806, 139, 1149, 172],
   [1163, 131, 1285, 171],
   [1297, 131, 1503, 171],
   [1517, 131, 1735, 162],
   [357, 229, 631, 262]],
  'ner_tags': [0, 1, 2,

In [77]:
train, test = train_test_split(final_list, random_state=21, test_size=0.3)

for detail  in final_list:
    with open('final_list_text.txt', 'a', encoding="utf-8") as f:
        f.write(str(detail))
        f.write('\n')
        
for detail  in train:
    with open('train.txt', 'a', encoding="utf-8") as f:
        f.write(str(detail))
        f.write('\n')
        
for detail  in test:
    with open('test.txt', 'a', encoding="utf-8") as f:
        f.write(str(detail))
        f.write('\n')




In [79]:
!pip install datasets

     ---------------------------------------- 0.0/474.3 kB ? eta -:--:--
     ----- --------------------------------- 61.4/474.3 kB 1.1 MB/s eta 0:00:01
     -------------------- ----------------- 256.0/474.3 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 474.3/474.3 kB 3.7 MB/s eta 0:00:00
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
     ---------------------------------------- 0.0/436.4 kB ? eta -:--:--
     ------------------------ ------------- 286.7/436.4 kB 8.6 MB/s eta 0:00:01
     -------------------------------------- 436.4/436.4 kB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/379.2 kB ? eta -:--:--
     ---------------------------------- -- 358.4/379.2 kB 11.2 MB/s eta 0:00:01
     -------------------------------------- 379.2/379.2 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/177.6 kB ? eta -:--:--
     ------------------------------------- 177.6/177.6 kB 10.5 MB/s eta 0:00:00
    


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
import json
import os
import ast
from pathlib import Path
import datasets
from PIL import Image
import pandas as pd

logger = datasets.logging.get_logger(__name__)
_CITATION = """\
@article{,
  title={},
  author={},
  journal={},
  year={},
  volume={}
}
"""
_DESCRIPTION = """\
This is a sample dataset for training layoutlmv3 model on custom annotated data.
"""

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    w, h = image.size
    return image, (w,h)

def normalize_bbox(bbox, size):
    return [
        int(1000 * bbox[0] / size[0]),
        int(1000 * bbox[1] / size[1]),
        int(1000 * bbox[2] / size[0]),
        int(1000 * bbox[3] / size[1]),
    ]


_URLS = []

'''Edit your working directory folder path here if required. 
If this file is in the same folder as the "layoutlmv3" folder keep it as it is.
'''
data_path = r'./' 

class DatasetConfig(datasets.BuilderConfig):
    """BuilderConfig for InvoiceExtraction Dataset"""
    def __init__(self, **kwargs):
        """BuilderConfig for InvoiceExtraction Dataset.
        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(DatasetConfig, self).__init__(**kwargs)


class InvoiceExtraction(datasets.GeneratorBasedBuilder):
    BUILDER_CONFIGS = [
        DatasetConfig(name="InvoiceExtraction", version=datasets.Version("1.0.0"), description="InvoiceExtraction dataset"),
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "bboxes": datasets.Sequence(datasets.Sequence(datasets.Value("int64"))),
                    "ner_tags": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names = ['invoice_no', 'date', 'amount'] #Enter the list of labels that you have here.
                            )
                    ),
                    "image_path": datasets.Value("string"),
                    "image": datasets.features.Image()
                }
            ),
            supervised_keys=None,
            citation=_CITATION,
            homepage="",
        )




    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        """Uses local files located with data_dir"""
        dest = os.path.join(data_path, 'layoutlmv3')

        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN, gen_kwargs={"filepath": os.path.join(dest, "train.txt"), "dest": dest}
            ),            
            datasets.SplitGenerator(
                name=datasets.Split.TEST, gen_kwargs={"filepath": os.path.join(dest, "test.txt"), "dest": dest}
            ),
        ]

    def _generate_examples(self, filepath, dest):

        df = pd.read_csv(os.path.join(dest, 'class_list.txt'), delimiter='\s', header=None)
        id2labels = dict(zip(df[0].tolist(), df[1].tolist()))


        logger.info("⏳ Generating examples from = %s", filepath)

        item_list = []
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                item_list.append(line.rstrip('\n\r'))
        print(item_list)
        for guid, fname in enumerate(item_list):
            print(fname)
            data = ast.literal_eval(fname)
            image_path = os.path.join(dest, data['file_name'])
            image, size = load_image(image_path)
            boxes = data['bboxes']

            text = data['tokens']
            label = data['ner_tags']
            
            #print(boxes)
            #for i in boxes:
            #  print(i)
            boxes = [normalize_bbox(box, size) for box in boxes]
            flag=0
            #print(image_path)
            for i in boxes:
              #print(i)
              for j in i:
                if j>1000:
                  flag+=1
                  #print(j)
                  pass
            if flag>0: print(image_path)
 
            yield guid, {"id": str(guid), "tokens": text, "bboxes": boxes, "ner_tags": label, "image_path": image_path, "image": image}

